In [1]:
# ========================================================================
# Cell 1: Import Libraries & Setup
# ========================================================================

import os
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
from torch.utils.tensorboard import SummaryWriter

from albumentations import Compose, Resize, Normalize, HorizontalFlip, RandomBrightnessContrast
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

from efficientnet_pytorch import EfficientNet

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Device: {device}")


✅ Device: cuda


In [2]:
# ========================================================================
# VERIFY YOUR SAVED MODEL
# ========================================================================

# Your previous experiment name
previous_experiment = "chest_xray_20251022_093109"  # From your training

# Check if model exists
model_path = f'experiments/{previous_experiment}/best_model.pth'

if os.path.exists(model_path):
    print("✅ GREAT NEWS! Your trained model was found!")
    print(f"   Location: {model_path}")
    
    # Load checkpoint info
    checkpoint = torch.load(model_path, weights_only=False)
    print(f"\n📊 Model Information:")
    print(f"   Trained for: {checkpoint['epoch']} epochs")
    print(f"   Validation AUC: {checkpoint['val_auc_macro']:.4f}")
    print(f"   Training complete: ✅")
    print("\n   YOU DON'T NEED TO RETRAIN!")
    
else:
    print("❌ Model not found!")
    print(f"   Looking for: {model_path}")
    print("\n   Please check the experiment folder name.")


✅ GREAT NEWS! Your trained model was found!
   Location: experiments/chest_xray_20251022_093109/best_model.pth

📊 Model Information:
   Trained for: 5 epochs
   Validation AUC: 0.8024
   Training complete: ✅

   YOU DON'T NEED TO RETRAIN!


In [3]:
# ========================================================================
# REBUILD MODEL AND LOAD TRAINED WEIGHTS
# ========================================================================

class ChestXrayClassifier(nn.Module):
    def __init__(self, num_classes=14, pretrained=True):
        super(ChestXrayClassifier, self).__init__()
        
        if pretrained:
            self.backbone = EfficientNet.from_pretrained('efficientnet-b0')
        else:
            self.backbone = EfficientNet.from_name('efficientnet-b0')
        
        num_features = self.backbone._fc.in_features
        self.backbone._fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(num_features, num_classes)
        )
    
    def forward(self, x):
        return self.backbone(x)

# Create model
model = ChestXrayClassifier(num_classes=14, pretrained=False)

# Load trained weights
checkpoint = torch.load(model_path, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
model = model.to(device)
model.eval()

print("="*70)
print("✅ MODEL LOADED SUCCESSFULLY!")
print("="*70)
print(f"Model: EfficientNet-B0")
print(f"Trained AUC: {checkpoint['val_auc_macro']:.4f}")
print(f"Epoch: {checkpoint['epoch']}")
print(f"Ready for: Next phase training")
print("="*70)


✅ MODEL LOADED SUCCESSFULLY!
Model: EfficientNet-B0
Trained AUC: 0.8024
Epoch: 5
Ready for: Next phase training


In [4]:
# ========================================================================
# PREPARE FOR 224x224 TRAINING (BETTER QUALITY)
# ========================================================================

print("\n" + "🚀 "*35)
print("PHASE 2: HIGH-RESOLUTION TRAINING (224x224)")
print("🚀 "*35 + "\n")

print("="*70)
print("WHY 224x224?")
print("="*70)
print("  • Your current model: 128x128 → 76.41% AUC")
print("  • Expected with 224x224: 78-79% AUC (+2-3%)")
print("  • Standard in medical imaging research")
print("  • Better detail preservation")
print("  • Still fits in 8GB GPU memory")
print("="*70)

# Load dataset paths
extract_path = "./chest_xray_data/"  # UPDATE if different
data_entry = pd.read_csv(os.path.join(extract_path, "Data_Entry_2017.csv"))

# Prepare splits (same as before)
train_list_path = os.path.join(extract_path, "train_val_list.txt")
test_list_path = os.path.join(extract_path, "test_list.txt")

if os.path.exists(train_list_path) and os.path.exists(test_list_path):
    train_list = pd.read_csv(train_list_path, header=None)[0].tolist()
    test_list = pd.read_csv(test_list_path, header=None)[0].tolist()
    train_val_data = data_entry[data_entry['Image Index'].isin(train_list)].reset_index(drop=True)
    test_data = data_entry[data_entry['Image Index'].isin(test_list)].reset_index(drop=True)
else:
    train_val_data, test_data = train_test_split(data_entry, test_size=0.3, random_state=42)

train_data, val_data = train_test_split(train_val_data, test_size=0.2, random_state=42)

print(f"\n✅ Dataset loaded:")
print(f"   Train:      {len(train_data):,} images")
print(f"   Validation: {len(val_data):,} images")
print(f"   Test:       {len(test_data):,} images")



🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 
PHASE 2: HIGH-RESOLUTION TRAINING (224x224)
🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 🚀 

WHY 224x224?
  • Your current model: 128x128 → 76.41% AUC
  • Expected with 224x224: 78-79% AUC (+2-3%)
  • Standard in medical imaging research
  • Better detail preservation
  • Still fits in 8GB GPU memory

✅ Dataset loaded:
   Train:      69,219 images
   Validation: 17,305 images
   Test:       25,596 images


In [5]:
# ========================================================================
# 224x224 TRANSFORMS (HIGHER RESOLUTION)
# ========================================================================

IMAGE_SIZE = 224  # INCREASED from 128

train_transforms_hires = Compose([
    Resize(IMAGE_SIZE, IMAGE_SIZE),
    HorizontalFlip(p=0.5),
    RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.3),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_transforms_hires = Compose([
    Resize(IMAGE_SIZE, IMAGE_SIZE),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

print("="*70)
print("HIGH-RESOLUTION CONFIGURATION")
print("="*70)
print(f"Image size:   {IMAGE_SIZE}x{IMAGE_SIZE} (was 128x128)")
print(f"Improvement:  3x more pixels per image")
print(f"Benefit:      Better detail, clearer pathology features")
print("="*70)


HIGH-RESOLUTION CONFIGURATION
Image size:   224x224 (was 128x128)
Improvement:  3x more pixels per image
Benefit:      Better detail, clearer pathology features


In [6]:
# ========================================================================
# DATASET CLASS (Same as before)
# ========================================================================

class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, num_classes=14):
        self.dataframe = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.num_classes = num_classes
        
        self.pathologies = [
            'Atelectasis', 'Consolidation', 'Infiltration', 
            'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 
            'Effusion', 'Pneumonia', 'Pleural_Thickening', 
            'Cardiomegaly', 'Nodule', 'Mass', 'Hernia'
        ]
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['Image Index']
        
        possible_paths = [
            os.path.join(self.img_dir, 'images', img_name),
            os.path.join(self.img_dir, img_name),
        ]
        
        for i in range(1, 13):
            possible_paths.append(
                os.path.join(self.img_dir, f'images_{i:03d}', 'images', img_name)
            )
        
        img_path = None
        for path in possible_paths:
            if os.path.exists(path):
                img_path = path
                break
        
        if img_path is None:
            raise FileNotFoundError(f"Image not found: {img_name}")
        
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)
        
        labels_str = self.dataframe.iloc[idx]['Finding Labels']
        labels = np.zeros(self.num_classes, dtype=np.float32)
        
        if labels_str != 'No Finding':
            for i, pathology in enumerate(self.pathologies):
                if pathology in labels_str:
                    labels[i] = 1.0
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image, torch.tensor(labels)

# ========================================================================
# CREATE DATALOADERS WITH 224x224 IMAGES
# ========================================================================

BATCH_SIZE = 32  # Reduced from 64 due to larger images
NUM_WORKERS = 0

train_dataset = ChestXrayDataset(train_data, img_dir=extract_path, transform=train_transforms_hires)
val_dataset = ChestXrayDataset(val_data, img_dir=extract_path, transform=val_transforms_hires)
test_dataset = ChestXrayDataset(test_data, img_dir=extract_path, transform=val_transforms_hires)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print(f"\n✅ DataLoaders created:")
print(f"   Image size:    {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"   Batch size:    {BATCH_SIZE} (reduced for memory)")
print(f"   Train batches: {len(train_loader):,}")
print(f"   Val batches:   {len(val_loader):,}")



✅ DataLoaders created:
   Image size:    224x224
   Batch size:    32 (reduced for memory)
   Train batches: 2,164
   Val batches:   541


In [7]:
# ========================================================================
# CALCULATE CLASS WEIGHTS
# ========================================================================

def calculate_class_weights(dataframe, num_classes=14):
    pathologies = [
        'Atelectasis', 'Consolidation', 'Infiltration', 
        'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 
        'Effusion', 'Pneumonia', 'Pleural_Thickening', 
        'Cardiomegaly', 'Nodule', 'Mass', 'Hernia'
    ]
    
    total_samples = len(dataframe)
    pos_weights = []
    
    for pathology in pathologies:
        pos_count = dataframe['Finding Labels'].str.contains(pathology, regex=False).sum()
        neg_count = total_samples - pos_count
        weight = neg_count / pos_count if pos_count > 0 else 1.0
        pos_weights.append(weight)
    
    return torch.tensor(pos_weights, dtype=torch.float32)

class_weights = calculate_class_weights(train_data)
class_weights = class_weights.to(device)

print("✅ Class weights calculated")


✅ Class weights calculated


In [8]:
# ========================================================================
# INITIALIZE NEW MODEL FOR HIGH-RESOLUTION TRAINING
# ========================================================================

# Create fresh model (will train from ImageNet weights again)
model_hires = ChestXrayClassifier(num_classes=14, pretrained=True)
model_hires = model_hires.to(device)

# Training setup
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights)
optimizer = AdamW(model_hires.parameters(), lr=0.0001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, min_lr=1e-7)
scaler = GradScaler()

# Training parameters
NUM_EPOCHS = 30
EARLY_STOPPING_PATIENCE = 7

# New experiment
experiment_name_hires = f"chest_xray_224x224_{time.strftime('%Y%m%d_%H%M%S')}"
os.makedirs(f'experiments/{experiment_name_hires}', exist_ok=True)
writer = SummaryWriter(f'experiments/{experiment_name_hires}/logs')

config_hires = {
    'model': 'EfficientNet-B0',
    'image_size': IMAGE_SIZE,
    'batch_size': BATCH_SIZE,
    'previous_model': '128x128 (76.41% AUC)',
    'target': '78-79% AUC',
    'improvement_expected': '+2-3%'
}

with open(f'experiments/{experiment_name_hires}/config.json', 'w') as f:
    json.dump(config_hires, f, indent=4)

print("="*70)
print("NEW MODEL INITIALIZED FOR 224x224 TRAINING")
print("="*70)
print(f"Previous model: 128x128 → 76.41% test AUC")
print(f"Target:         224x224 → 78-79% test AUC")
print(f"Expected gain:  +2-3% AUC")
print(f"Training time:  ~20-24 hours")
print("="*70)


Loaded pretrained weights for efficientnet-b0
NEW MODEL INITIALIZED FOR 224x224 TRAINING
Previous model: 128x128 → 76.41% test AUC
Target:         224x224 → 78-79% test AUC
Expected gain:  +2-3% AUC
Training time:  ~20-24 hours


C:\Users\91850\AppData\Local\Temp\ipykernel_20112\2575219465.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [9]:
# ========================================================================
# TRAINING FUNCTIONS
# ========================================================================

def train_epoch(model, loader, criterion, optimizer, device, epoch, scaler):
    model.train()
    running_loss = 0.0
    all_labels = []
    all_predictions = []
    
    print(f"\n🔄 Training Epoch {epoch+1}")
    epoch_start = time.time()
    
    for batch_idx, (images, labels) in enumerate(loader):
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)
        
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item()
        
        if batch_idx % 5 == 0:
            all_labels.append(labels.cpu().numpy())
            predictions = torch.sigmoid(outputs).detach().cpu().numpy()
            all_predictions.append(predictions)
        
        if (batch_idx + 1) % 50 == 0:
            avg_loss = running_loss / (batch_idx + 1)
            elapsed = time.time() - epoch_start
            speed = (batch_idx + 1) / elapsed
            eta = (len(loader) - batch_idx - 1) / speed / 60
            print(f"  [{batch_idx+1}/{len(loader)}] Loss: {avg_loss:.4f} | "
                  f"{speed:.2f} batch/s | ETA: {eta:.1f}min")
    
    if len(all_labels) > 0:
        all_labels = np.vstack(all_labels)
        all_predictions = np.vstack(all_predictions)
        avg_loss = running_loss / len(loader)
        try:
            auc_macro = roc_auc_score(all_labels, all_predictions, average='macro')
            auc_weighted = roc_auc_score(all_labels, all_predictions, average='weighted')
        except:
            auc_macro = 0.0
            auc_weighted = 0.0
    else:
        avg_loss = running_loss / len(loader)
        auc_macro = 0.0
        auc_weighted = 0.0
    
    return avg_loss, auc_macro, auc_weighted


def validate_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_predictions = []
    
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            all_labels.append(labels.cpu().numpy())
            predictions = torch.sigmoid(outputs).cpu().numpy()
            all_predictions.append(predictions)
    
    all_labels = np.vstack(all_labels)
    all_predictions = np.vstack(all_predictions)
    avg_loss = running_loss / len(loader)
    
    auc_macro = roc_auc_score(all_labels, all_predictions, average='macro')
    auc_weighted = roc_auc_score(all_labels, all_predictions, average='weighted')
    
    per_class_auc = []
    for i in range(all_labels.shape[1]):
        try:
            auc = roc_auc_score(all_labels[:, i], all_predictions[:, i])
            per_class_auc.append(auc)
        except:
            per_class_auc.append(0.0)
    
    return avg_loss, auc_macro, auc_weighted, per_class_auc

print("✅ Training functions ready")


✅ Training functions ready


In [10]:
# ========================================================================
# START 224x224 HIGH-RESOLUTION TRAINING
# ========================================================================

def train_model_hires():
    best_val_auc = 0.0
    early_stop_counter = 0
    training_start_time = time.time()
    
    pathologies = [
        'Atelectasis', 'Consolidation', 'Infiltration', 
        'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 
        'Effusion', 'Pneumonia', 'Pleural_Thickening', 
        'Cardiomegaly', 'Nodule', 'Mass', 'Hernia'
    ]
    
    print("\n" + "="*70)
    print("🚀 STARTING 224x224 HIGH-RESOLUTION TRAINING")
    print("="*70)
    print(f"Previous model (128x128): 76.41% test AUC")
    print(f"Target (224x224):         78-79% test AUC")
    print(f"Expected training time:   20-24 hours")
    print("="*70)
    
    for epoch in range(NUM_EPOCHS):
        epoch_start = time.time()
        
        print(f"\n{'='*70}")
        print(f"EPOCH [{epoch+1}/{NUM_EPOCHS}]")
        print(f"{'='*70}")
        
        # Train
        train_loss, train_auc_macro, train_auc_weighted = train_epoch(
            model_hires, train_loader, criterion, optimizer, device, epoch, scaler
        )
        
        # Validate
        val_loss, val_auc_macro, val_auc_weighted, per_class_auc = validate_epoch(
            model_hires, val_loader, criterion, device
        )
        
        # Update LR
        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_auc_macro)
        current_lr = optimizer.param_groups[0]['lr']
        
        if old_lr != current_lr:
            print(f"\n📉 LR: {old_lr:.2e} → {current_lr:.2e}")
        
        epoch_time = time.time() - epoch_start
        total_elapsed = (time.time() - training_start_time) / 3600
        
        # Log
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('AUC/train', train_auc_macro, epoch)
        writer.add_scalar('AUC/val', val_auc_macro, epoch)
        writer.add_scalar('LR', current_lr, epoch)
        
        # Print
        print(f"\n📊 Results:")
        print(f"  Train: Loss={train_loss:.4f}, AUC={train_auc_macro:.4f}")
        print(f"  Val:   Loss={val_loss:.4f}, AUC={val_auc_macro:.4f}")
        print(f"  Time: {epoch_time/60:.1f}min | Total: {total_elapsed:.2f}hrs")
        
        # Compare with previous model
        improvement_vs_128 = val_auc_macro - 0.8024  # Your 128x128 val AUC
        print(f"  vs 128x128: {improvement_vs_128:+.4f} ({improvement_vs_128*100:+.2f}%)")
        
        if epoch > 0:
            avg_time = (time.time() - training_start_time) / (epoch + 1)
            eta = (avg_time * (NUM_EPOCHS - epoch - 1)) / 3600
            print(f"  ETA: {eta:.2f} hours")
        
        # Save best
        if val_auc_macro > best_val_auc:
            best_val_auc = val_auc_macro
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model_hires.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_auc_macro': val_auc_macro,
                'per_class_auc': per_class_auc,
                'config': config_hires
            }, f'experiments/{experiment_name_hires}/best_model.pth')
            print(f"\n  ✅ Best model saved! AUC: {best_val_auc:.4f}")
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            print(f"\n  ⏳ No improvement ({early_stop_counter}/{EARLY_STOPPING_PATIENCE})")
        
        if early_stop_counter >= EARLY_STOPPING_PATIENCE:
            print(f"\n⚠️ Early stopping at epoch {epoch+1}")
            break
    
    total_time = time.time() - training_start_time
    print("\n" + "="*70)
    print("✅ 224x224 TRAINING COMPLETE!")
    print(f"Best AUC: {best_val_auc:.4f}")
    print(f"Time: {total_time/3600:.2f} hours")
    print(f"Improvement over 128x128: {(best_val_auc-0.8024)*100:+.2f}%")
    print("="*70)
    
    writer.close()
    return best_val_auc

# START TRAINING
print(f"\n⏰ Starting: {time.strftime('%I:%M %p IST')}")
print(f"⏰ Expected: ~{time.strftime('%I:%M %p', time.localtime(time.time() + 86400))} IST (tomorrow)")

best_auc_hires = train_model_hires()



⏰ Starting: 10:12 AM IST
⏰ Expected: ~10:12 AM IST (tomorrow)

🚀 STARTING 224x224 HIGH-RESOLUTION TRAINING
Previous model (128x128): 76.41% test AUC
Target (224x224):         78-79% test AUC
Expected training time:   20-24 hours

EPOCH [1/30]

🔄 Training Epoch 1


C:\Users\91850\AppData\Local\Temp\ipykernel_20112\2896649790.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


  [50/2164] Loss: 1.3224 | 0.35 batch/s | ETA: 101.4min
  [100/2164] Loss: 1.3472 | 0.35 batch/s | ETA: 99.1min
  [150/2164] Loss: 1.3238 | 0.35 batch/s | ETA: 96.2min
  [200/2164] Loss: 1.3261 | 0.35 batch/s | ETA: 93.7min
  [250/2164] Loss: 1.3257 | 0.35 batch/s | ETA: 91.0min
  [300/2164] Loss: 1.3332 | 0.35 batch/s | ETA: 88.7min
  [350/2164] Loss: 1.3169 | 0.35 batch/s | ETA: 85.8min
  [400/2164] Loss: 1.3117 | 0.35 batch/s | ETA: 84.0min
  [450/2164] Loss: 1.3042 | 0.34 batch/s | ETA: 84.5min
  [500/2164] Loss: 1.2902 | 0.34 batch/s | ETA: 80.5min
  [550/2164] Loss: 1.2862 | 0.36 batch/s | ETA: 75.4min
  [600/2164] Loss: 1.2688 | 0.37 batch/s | ETA: 70.8min
  [650/2164] Loss: 1.2740 | 0.38 batch/s | ETA: 67.0min
  [700/2164] Loss: 1.2699 | 0.39 batch/s | ETA: 63.3min
  [750/2164] Loss: 1.2542 | 0.39 batch/s | ETA: 60.0min
  [800/2164] Loss: 1.2461 | 0.40 batch/s | ETA: 56.9min
  [850/2164] Loss: 1.2322 | 0.41 batch/s | ETA: 53.9min
  [900/2164] Loss: 1.2318 | 0.41 batch/s | ETA: 

C:\Users\91850\AppData\Local\Temp\ipykernel_20112\2896649790.py:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():



📊 Results:
  Train: Loss=1.1889, AUC=0.7433
  Val:   Loss=1.1609, AUC=0.7956
  Time: 92.0min | Total: 1.53hrs
  vs 128x128: -0.0068 (-0.68%)

  ✅ Best model saved! AUC: 0.7956

EPOCH [2/30]

🔄 Training Epoch 2


...


📊 Results:
  Train: Loss=0.5398, AUC=0.9291
  Val:   Loss=2.1835, AUC=0.8070
  Time: 65.0min | Total: 14.09hrs
  vs 128x128: +0.0046 (+0.46%)
  ETA: 28.18 hours

  ⏳ No improvement (6/7)

EPOCH [11/30]

🔄 Training Epoch 11


C:\Users\91850\AppData\Local\Temp\ipykernel_20112\2896649790.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


  [50/2164] Loss: 0.4736 | 0.48 batch/s | ETA: 72.9min
  [100/2164] Loss: 0.4768 | 0.47 batch/s | ETA: 72.5min
  [150/2164] Loss: 0.4812 | 0.47 batch/s | ETA: 71.0min
  [200/2164] Loss: 0.4845 | 0.47 batch/s | ETA: 69.6min
  [250/2164] Loss: 0.4876 | 0.47 batch/s | ETA: 68.2min
  [300/2164] Loss: 0.4974 | 0.46 batch/s | ETA: 67.5min
  [350/2164] Loss: 0.4922 | 0.45 batch/s | ETA: 67.1min
  [400/2164] Loss: 0.4972 | 0.44 batch/s | ETA: 66.1min
  [450/2164] Loss: 0.4986 | 0.44 batch/s | ETA: 64.8min
  [500/2164] Loss: 0.4983 | 0.44 batch/s | ETA: 62.6min
  [550/2164] Loss: 0.4955 | 0.44 batch/s | ETA: 61.3min
  [600/2164] Loss: 0.4967 | 0.44 batch/s | ETA: 59.6min
  [650/2164] Loss: 0.4960 | 0.45 batch/s | ETA: 56.2min
  [700/2164] Loss: 0.4972 | 0.46 batch/s | ETA: 53.0min
  [750/2164] Loss: 0.4986 | 0.47 batch/s | ETA: 50.0min
  [800/2164] Loss: 0.4969 | 0.48 batch/s | ETA: 47.2min
  [850/2164] Loss: 0.4999 | 0.49 batch/s | ETA: 44.6min
  [900/2164] Loss: 0.4984 | 0.50 batch/s | ETA: 4

C:\Users\91850\AppData\Local\Temp\ipykernel_20112\2896649790.py:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():



📊 Results:
  Train: Loss=0.5089, AUC=0.9337
  Val:   Loss=2.4142, AUC=0.8089
  Time: 72.4min | Total: 15.30hrs
  vs 128x128: +0.0065 (+0.65%)
  ETA: 26.42 hours

  ⏳ No improvement (7/7)

⚠️ Early stopping at epoch 11

✅ 224x224 TRAINING COMPLETE!
Best AUC: 0.8265
Time: 15.30 hours
Improvement over 128x128: +2.41%
